# Explore Colenda Items Over Time

[Colenda Digital Repository at Penn Libraries](https://colenda.library.upenn.edu/) is a digital repository for digitized and born-digital material. It provides direct access and long-term stewardship for these important resources. Much of Colenda’s content consists of materials owned and digitized by the Penn Libraries, including significant collections that have been donated.

In this notebook we'll explore the temporal dimensions of data harvested from Colenda. When were items created, collected, or used? To do that we'll extract the nested temporal data, see what's there, and create a few charts.

[See here](kaplan_explore_records.ipynb) for an introduction to exploring Colenda data, and [here to explore spatial dimensions](kaplan_explore_places.ipynb) of the data.

* [Import What We Need](#Import-What-We-Need)
* [Load the Data](#Load-the-Data)
* [Concatenate and Split `metadata.date` Fields](#Concatenate-and-Split-metadata.date-Fields)
* [Explore Items by Year](#Explore-Items-By-Year)
* [Explore Items Distributed Over Time](#Explore-Items-Distributed-Over-Time)
* [Focus on Items in a Particular Year](#Focus-on-Items-in-a-Particular-Year)
* [Explore Items by Year and Type](#Explore-Items-by-Year-and-Type)
* [Explore Items by Year and Location](#Explore-Items-by-Year-and-Location)
* [Need Help?](#Need-Help?)
* [Credits](#Credits)

<div class="alert alert-block alert-warning">
<p><b>Yellow blocks like this provide additional information about Python and Jupyter notebooks.</b></p>
    
<p>If you haven't used one of these notebooks before, they're basically web pages in which you can write, edit, and run live code. They're meant to encourage experimentation, so don't feel nervous. Just try running a few cells and see what happens!</p>

<p>
    Some tips:
    <ul>
        <li>Code cells have boxes around them.</li>
        <li>To run a code cell click on the cell and then hit <b>Shift+Enter</b>. The <b>Shift+Enter</b> combo will also move you to the next cell, so it's a quick way to work through the notebook.</li>
        <li>While a cell is running a <b>*</b> appears in the square brackets next to the cell. Once the cell has finished running the asterix will be replaced with a number.</li>
        <li>In most cases you'll want to start from the top of notebook and work your way down running each cell in turn. Later cells might depend on the results of earlier ones.</li>
        <li>To edit a code cell, just click on it and type stuff. Remember to run the cell once you've finished editing.</li>
    </ul>
</p>

<p><b>Is this thing on?</b> If you can't edit or run any of the code cells, you might be viewing a static (read only) version of this notebook. Click here to <a href="https://mybinder.org/v2/gh/GLAM-Workbench/national-museum-australia/master?urlpath=lab%2Ftree%2Fexplore_collection_object_over_time.ipynb">load a <b>live</b> version</a> running on Binder.</p>
</div>

## Import What We Need

<div>
    <p>In order to use this notebook, you first need to `import` modules and packages from Python. These are units of code with specific tools or skills that we use in the script.</p>
<div class="alert alert-block alert-warning">
<p>If you're running this notebook on your computer, you may need to first `import` these modules within your Python interpreter. Find assistance for that <a href="https://packaging.python.org/tutorials/installing-packages/">here</a>.</p>
    </div>

In [1]:
!pip install -r requirements.txt

# Pandas is a Python package that provides numerous tools for data analysis. 
import pandas as pd

# IpyLeaflet is a Python library that enables interactive geospatial data visualization in Jupyter Notebook.
from ipyleaflet import Map, Marker, Popup, MarkerCluster

# IpyWidgets is a Python library that enables interactive HTML widgets for Jupyter notebooks.
import ipywidgets as widgets

# Altair is a Python library for declarative statistical visualization. 
import altair as alt

# IPython is a Python interpreter to display content.
from IPython.display import display, HTML, FileLink

     |████████████████████████████████| 3.3 MB 4.3 MB/s eta 0:00:01
  Using cached geopandas-0.9.0-py2.py3-none-any.whl (994 kB)
  Using cached statistics-1.0.3.5.tar.gz (8.3 kB)
     |████████████████████████████████| 7.6 MB 6.6 MB/s eta 0:00:01
     |████████████████████████████████| 19.3 MB 9.4 MB/s eta 0:00:01
     |████████████████████████████████| 1.0 MB 5.1 MB/s eta 0:00:01
  Using cached cligj-0.7.2-py3-none-any.whl (7.1 kB)
  Using cached click_plugins-1.1.1-py2.py3-none-any.whl (7.5 kB)
  Using cached munch-2.5.0-py2.py3-none-any.whl (10 kB)
  Using cached traittypes-0.2.1-py2.py3-none-any.whl (8.6 kB)


  Created wheel for statistics: filename=statistics-1.0.3.5-py3-none-any.whl size=7454 sha256=75bb8c6028f3ecf51e770c426c4bbfcb3e78cbc94aecd0369f2a0a714a2ff5ca
  Stored in directory: /Users/estene/Library/Caches/pip/wheels/36/4b/c7/6af97584669b756c0d60c5ff05d5fb1f533a4e4d96e5ee92b9
Successfully built statistics


## Load the Data

This pre-harvested dataset from Colenda includes every gift of Arnold and Deanne Kaplan, which covers two collections. In this notebook we will only work with records from the Arnold and Deanne Kaplan Collection of **Early American Judaica**. Let's load our data into Python using [Pandas](https://www.w3schools.com/python/pandas/default.asp), and then filtering on the Early American Judaica collection  by using the `metadata.collection[1]` column. 

In [2]:
# Convert to a dataframe (a two-dimensional data structure with rows and columns)
df = pd.read_csv("data/kaplan-test-data.csv", encoding= 'unicode_escape')

# Print the number of rows in the dataframe
print('There are {:,} items in this dataset from Colenda.'.format(df.shape[0]))

There are 9,418 items in this dataset from Colenda.


/Users/estene/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (2,7,10,16,23,24,25,26,27,28,29,30,31,32,33,44,45,46,47,48,49,53,54,55,56,57,58,61,62,63,64,65,66,67,68,69,70,72,76,77,78,79,80,82,84,85,86,95) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


<div class="alert alert-block alert-warning">
<p>You may see a warning appear above stating that columns having "mixed types". This means that the CSV columns contain a mix of strings and integers. When converting the CSV into a dataframe, Python wasn't sure how to declare the column type. It's OK to ignore this warning for now - we may need to state directly this information later.<p>
</div>

In [3]:
# Filter for items from the Early American Judaica Collection. 
df = df.loc[df['metadata.collection[1]'] == "Arnold and Deanne Kaplan Collection of Early American Judaica (University of Pennsylvania)"]

# Return the first 5 rows of the dataframe.
df.head()

,metadata.call_number[1],metadata.collection[1],metadata.contributor[1],metadata.corporate_name[1],metadata.corporate_name[2],metadata.corporate_name[3],metadata.corporate_name[4],metadata.corporate_name[5],metadata.corporate_name[6],metadata.date[1],...,metadata.subject[2],metadata.subject[3],metadata.subject[4],metadata.subject[5],metadata.subject[6],metadata.subject[7],metadata.subject[8],metadata.subject[9],metadata.title[1],unique_identifier
2,Arc.MS.56,Arnold and Deanne Kaplan Collection of Early A...,NaN,Baum & Bernstein,NaN,NaN,NaN,NaN,NaN,unknown,...,Jewish merchants,Trade cards (advertising),NaN,NaN,NaN,NaN,NaN,NaN,"Trade card; Baum & Bernstein; Meriden, Connect...",ark:/81431/p3000003f
3,Arc.MS.56,Arnold and Deanne Kaplan Collection of Early A...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1837-11-07,...,Jewish merchants,Family papers,Manuscripts (documents),NaN,NaN,NaN,NaN,NaN,"Letter; Tobias, Henry; Liverpool, United Kingd...",ark:/81431/p3000006w
4,Arc.MS.56,Arnold and Deanne Kaplan Collection of Early A...,NaN,I. H. Brounstein's One Price Clothing House,NaN,NaN,NaN,NaN,NaN,unknown,...,Advertisements,Jewish merchants,Clothing trade,NaN,NaN,NaN,NaN,NaN,Trade card; I. H. Brounstein's One Price Cloth...,ark:/81431/p3000013w
5,Arc.MS.56,Arnold and Deanne Kaplan Collection of Early A...,NaN,Kast's Fine Shoes,NaN,NaN,NaN,NaN,NaN,unknown,...,Jewish merchants,Trade cards (advertising),NaN,NaN,NaN,NaN,NaN,NaN,"Trade card; Kast's Fine Shoes; San Francisco, ...",ark:/81431/p3000019s
6,Arc.MS.56,Arnold and Deanne Kaplan Collection of Early A...,NaN,Mandel Bro's,NaN,NaN,NaN,NaN,NaN,unknown,...,General stores,Dry-goods,Clothing trade,Jewish merchants,Trade cards (advertising),NaN,NaN,NaN,Trade card; Mandel Bro's; undated,ark:/81431/p3000020w


## Concatenate and Split `metadata.date` Fields

Specific dates or years are linked to items through the `metadata.date` columns. Each item may have multiple multiple dates associated with it. For comparative and quantitative data analysis, we need to split those items into multiple rows instead of columns.

We'll write two **functions** help us do that: 
* `tidy_split` splits the values of each cell on a "|" so that there is one split value per row, and 
* `tidy_concat` concatenates (joins) the values of columns that begin with a similar phrase into one cell with a "|" before using `tidy_split`. 

Now instead of having one row for each item with multiple dates, we can have one row for each date associated with an item.

<div class="alert alert-block alert-warning">
<p>A function is a block of reusable code that is used to perform a single, related action. Learn more about functions <a href="https://www.w3schools.com/python/python_functions.asp">here</a>).</p>
</div>

In [4]:
# Split the values of a column and expand so that the new DataFrame has one split value per row. Filters rows where column is empty. 
def tidy_split(df, column, sep='|', keep=False):
    """
    Params
    ------
    df : pandas.DataFrame
        dataframe with the column to split and expand
    column : str
        the column to split and expand
    sep : str
        the string used to split the column's values
    keep : bool
        whether to retain the presplit value as its own row

    Returns
    -------
    pandas.DataFrame
        Returns a dataframe with the same columns as `df`.
    """
    indexes = list()
    new_values = list()
    df = df.dropna(subset=[column])
    for i, presplit in enumerate(df[column].astype(str)):
        values = presplit.split(sep)
        if keep and len(values) > 1:
            indexes.append(i)
            new_values.append(presplit)
        for value in values:
            indexes.append(i)
            new_values.append(value)
    new_df = df.iloc[indexes, :].copy()
    new_df[column] = new_values
    return new_df

In [5]:
# Concatenate the values of columns beginnigng with a string and then use the tidy_split function to expand so that the new DataFrame has one split value per row.
def tidy_concat(df, column_starts_with, sep="|"):
    """
    Params
    ------
    df : pandas.DataFrame
        dataframe with the columns to split and expand
    column_starts_with : str
        the string at the beginning of the column(s) to split
    sep : str
        the string used to split the column's values

    Returns
    -------
    pandas.DataFrame
        Returns a dataframe with the same columns as `df`.
    """
    list_of_columns = df.columns.to_list()
    columns_to_concat = [x for x in list_of_columns if x.startswith(column_starts_with)]
    df[column_starts_with] = df[columns_to_concat[0]]
    for column in columns_to_concat[1:]:
        df[column_starts_with] = df[column_starts_with].astype(str) + sep + df[column].astype(str)
    new_df = tidy_split(df, column_starts_with, sep='|')
    new_df = new_df.drop(columns_to_concat, axis=1)
    return new_df

Now that we've defined these the `tidy_concat` and `tidy_split` functions, we can use them to concatenate the multiple `metadata.date` columns into one column, and then split it so that the new dataframe has one split vavlue per row.

In [6]:
# Concatenate the values of columns beginnigng with a string and then use the tidy_split function to expand so that the new DataFrame has one split value per row.
df_dates = tidy_concat(df, 'metadata.date', sep='|')

# Filter the dataframe to only include dates that are NOT 'unknown'
df_dates = df_dates[df_dates["metadata.date"]!='unknown']

# Filter the datafarme to only include the dates are NOT 'nan' (not a number)
df_dates = df_dates[df_dates["metadata.date"]!='nan']

# Report the dimensionality of the dataframe (number of rows, number of columns).
df_dates.shape

(4532, 96)



How many **unique** dates are represented in the collection?

In [7]:
# Drop the duplicate dates from the dataframe and report the dimensionality of the dataframe.
unique_df_dates = df_dates.drop_duplicates(subset=['metadata.date']).shape

print('There are {:,} unique dates represented in the Arnold and Deanne Kaplan Collection of Early American Judaica (University of Pennsylvania).'.format(unique_df_dates[0]))

There are 3,391 unique dates represented in the Arnold and Deanne Kaplan Collection of Early American Judaica (University of Pennsylvania).


## Explore Items by Year

Dates in this Colenda are formatted YYYY-MONTH-DD for a specific date. Let's extract years from the dates to make comparisons a bit easier.

In [8]:
# Use a regular expression (a specific text pattern) to find the first four digits in the date fields
df_dates['metadata.year'] = df_dates['metadata.date'].str.extract(r'^(\d{4})').fillna(0).astype('int')

<div class="alert alert-block alert-warning">
<p>A regular expressions is a sequence of characters that forms a search pattern, often used to find if a string contains a specific search pattern. Find more information about regular expression <a href="https://www.w3schools.com/python/python_regex.asp">here</a>.</p>
</div>

What's the earliest `year` represented in the collection?

In [9]:
# Locate the minimum year (above 0) represented in the dataframe's Year column. 
earliest_year = df_dates.loc[df_dates['metadata.year'] > 0]['metadata.year'].min()

print(earliest_year)

1555


What items are from 1555? 

To identify these items, let's create a dataframe of items from 1555 and Then, we'll use the item's ARK identifier to generate links to an item's record in Colenda. More about ARK identifiers can be found in [Explore Colenda Records](kaplan_explore_records.ipynb). 

In [10]:
# Create a dataframe called df_earliest, which is a subsection of the df_dates dataframe that contains only those where the year = earliest_year
df_earliest = df_dates[df_dates['metadata.year'] == earliest_year]

# Select the first row in the dataframe and access the Unique Identifier
identifier = df_earliest.iloc[0]['unique_identifier']

# Split the string up to the second occurrence of "/" and join all but the first element of the split string 
identifier = "-".join(identifier.split("/", 2)[1:])

# Display the link to the item in Colenda, with the item-specific URL and the item's title as the hyperlinked text 
display(HTML('<a href="https://colenda.library.upenn.edu/catalog/{}">{}</a>'.format(identifier, df_earliest.iloc[0]['metadata.title[1]'])))

What's the latest year represented in the collection?

In [11]:
# Return the maximum year of the years represented in the dataframe
latest_year = df_dates['metadata.year'].max()

print(latest_year)

1905


How many items are there from the latest year? 

In [12]:
# Create a dataframe called df_latest, which is a subsection of the df_dates dataframe that contains only those where the year = latest_year
df_latest = df_dates[df_dates['metadata.year'] == latest_year]

print('There are {:,} items from 1899 represented in the collection'.format(df_latest.shape[0]))

There are 9 items from 1899 represented in the collection


The year `1899` is used as a boundary year between two Kaplan Collections at the University of Pennsylvania Libraries: that of **Early American Judaica** (pre-1899) and **Modern American Judaica** (post-1899). What items are from 1899? 

Since there may be multiple items from 1899, we'll use a *for* loop to iterate over all the items from that year. 

<div class="alert alert-block alert-warning">
    <p>A <i>for</i> loop is used when iterating over a sequence in order. For each item in the sequence, the same action(s) will be performed. Find more information on <i>for</i> loops <a href="https://www.w3schools.com/python/python_for_loops.asp">here</a></p>
</div>

In [13]:
count = 0

# Create a for loop that iterates over rows in the df_latest dataframe
for idx, row in df_latest.iterrows():
    
# Select the first row in the dataframe and access the Unique Identifier 
    identifier = df_latest.iloc[count]['unique_identifier']

# Split the string up to the second occurrence of "/" and join all but the first two elements of the split string 
    identifier = "-".join(identifier.split("/", 2)[1:])

# Display the link to the item in Colenda, with the item-specific URL and the item's title as the hyperlinked text
    display(HTML('<a href="https://colenda.library.upenn.edu/catalog/{}">{}</a>'.format(identifier, df_latest.iloc[count]['metadata.title[1]'])))
    count+=1

## Explore Items Distributed Over Time

Let's use the `Statistics` module to gather additional information about the years represented in the collection.

In [14]:
#Statistics is a Python module that you can use to calculate mathematical statistics of numeric data
import statistics 

# Convert the values of the Year column to a list
col_dates_list = df_dates['metadata.year'].tolist()

# Remove all the 0 values from the list. 
col_dates_list = [i for i in col_dates_list if i != 0]

# Sorted is a Python method that returns a numerically sorted list
sorted(col_dates_list)

# Calculate the median year in the list 
median = statistics.median(col_dates_list)

# Calculate the mode year in the list
mode = statistics.mode(col_dates_list)

print('The median year is ' + str(median) + ' for items in the collection.')
print('The most common year is ' + str(mode) + ' for items in the collection.')

The median year is 1863.0 for items in the collection.
The most common year is 1881 for items in the collection.


The median and mode suggest that the majority of items in this collection are associated with the late nineteenth century. Let's make a chart of to see the number of items per year to see this as a visualization. 

In [15]:
# Count how many rows contain a year in the Year column. and save as a dataframe called year_counts
year_counts = df_dates['metadata.year'].value_counts().to_frame().reset_index()
year_counts.columns = ['Year', 'Count']
year_counts.head(10)

,Year,Count
0,1881,156
1,1885,136
2,1882,125
3,1884,117
4,1825,107
5,1880,104
6,1883,96
7,1875,93
8,1886,86
9,1877,84


In [16]:
# Create a bar chart (limit to years greater than 0)
alt.Chart(year_counts.loc[year_counts['Year'] > 0]).mark_bar(size=2).encode(
    
    # Year on the X axis
    x=alt.X('Year:Q', axis=alt.Axis(format='c', title='Year')),
    
    # Number of objects on the Y axis
    y=alt.Y('Count:Q', title='Number of Items'),
    
    # Show details on hover
    tooltip=[alt.Tooltip('Year:Q', title='Year'), alt.Tooltip('Count:Q', title='Count', format=',')]
).properties(width=700)

alt.Chart(...)

## Focus on Items in a Particular Year

In another notebook, we [used the `metadata.item_type` columns](kaplan_explore_records.ipynb#The-metadata.item_type-Fields) to learn about the types of items in the collection. Let's use it to see what types of objects are associated with a particular year. For the Kaplan Collection, we will use the year 1881.

Let's explode `metadata.item_type[1]` and create a new dataframe with the results.

In [17]:
# Identify the rows where the `metadata.item_type[1]` column is not null, and create a new dataframe that maintains four of the columns 
df_dates_types = df_dates.loc[df_dates['metadata.item_type[1]'].notnull()][['unique_identifier', 'metadata.title[1]', 'metadata.year', 'metadata.item_type[1]']]

# Return the first 5 lines of the df_dates_types dataframe
df_dates_types.head()

,unique_identifier,metadata.title[1],metadata.year,metadata.item_type[1]
3,ark:/81431/p3000006w,"Letter; Tobias, Henry; Liverpool, United Kingd...",1837,Letters
9,ark:/81431/p3000023b,"Trade card; Erichs'; New York, New York, Unite...",1881,Trade cards
14,ark:/81431/p30000507,"Billhead; Jacob Hyman & Son; New York, New Yor...",1886,Billheads
15,ark:/81431/p3000051q,"Letter; Janison, Samuel; Levy, Louis E.; Jewis...",1890,Letters
16,ark:/81431/p30000526,"Receipt; Lopez, Aaron; Vernon, William; Newpor...",1763,Receipts


Now we can filter by year to see what types of items are associated in 1881.

In [18]:
# Create a new dataframe from the df_dates_types dataframe of the rows where the year is 1881. 
year_1881 = df_dates_types.loc[df_dates_types['metadata.year'] == 1881]
year_1881.head()

,unique_identifier,metadata.title[1],metadata.year,metadata.item_type[1]
9,ark:/81431/p3000023b,"Trade card; Erichs'; New York, New York, Unite...",1881,Trade cards
25,ark:/81431/p3022f,"Billhead; Meyer, M.; M. Meyer & Co.; Selma, Al...",1881,Billhead
52,ark:/81431/p3053v,Billhead; Lisberger; The Lisburger & Wise Pant...,1881,Billhead
62,ark:/81431/p3057cv04,"Trade card; S. Rosenbloom & Son; Auburn, New Y...",1881,Trade cards
80,ark:/81431/p3082h,"Billhead; Julius Knoesel; New York, New York, ...",1881,Billhead


Let's look at the top twenty-five types of things created in 1881.

In [19]:
# Return a Series containing counts of unique rows in the dataframe for each item type (up to 25)
year_1881['metadata.item_type[1]'].value_counts()[:25]

Trade cards           77
Billhead              28
Billheads             14
Letters                9
Letterheads            5
Periodicals            4
Deeds                  3
Broadsides             2
Monetary               2
Envelopes              2
Pamphlets              1
Court records          1
Drawings               1
Advertisments          1
Receipts               1
Invitations            1
Newspaper              1
Official documents     1
Ports of entry         1
Tickets                1
Name: metadata.item_type[1], dtype: int64

The vast majority of items are 'Trade cards'. Let's look at one of the 'Trade cards' in more detail.

The images in Colenda for these Trade cards are available under the [**International Image Interoperability Framework (IIIF)**](https://iiif.io/), which makes these images accessible and interoperable between image repositories. 
Let's take a look at the images of the clock. 

To display those images, we'll write two **functions** help us do that: 
* `_src_from_data` splits the values of each cell on a "|" so that there is one split value per row, and 
* `gallery` shows a set of images in a gallery that flexes with the width of the notebook. 

These functions for working with IIIF images come from [BVMC Labs](http://data.cervantesvirtual.com/). 

In [20]:
# Encode image bytes for inclusion in an HTML img element
# Requests is a Python package that allows you to send HTTP/1.1 requests.
import requests

def _src_from_data(data):
    img_obj = Image(data=data)
    for bundle in img_obj._repr_mimebundle_():
        for mimetype, b64value in bundle.items():
            if mimetype.startswith('image/'):
                return f'data:{mimetype};base64,{b64value}'

#  Shows a set of images in a gallery that flexes with the width of the notebook.
def gallery(dictionary, row_height='auto'):
    figures = []
    for image, label in dictionary.items():
        src = image
        figures.append(f'''<figure style="margin: 5px !important;">
        <img src="{src}" 
        style="height: {row_height}">
        <figcaption style="font-size: 1em">{label}</figcaption>
        </figure>''')
    return HTML(data=f'''<div style="display: flex; flex-flow: row wrap; text-align: center;">{''.join(figures)}</div>''')

In [21]:
# Locate in the year_1881 dataframe of the rows where the type is Trade cards
year_1881.loc[year_1881['metadata.item_type[1]'] == 'Trade cards'].head()

# Select the first row in the dataframe. 
item = year_1881.loc[year_1881['metadata.item_type[1]'] == 'Trade cards'].iloc[0]

# Select the first row in the dataframe and access the Unique Identifier. 
identifier = year_1881.iloc[count]['unique_identifier']

# Split the string up to the second occurrence of "/" and join all but the first two elements of the split string. 
identifier = "-".join(identifier.split("/", 2)[1:])

# Create a string that is the link to the item-specific IIIF manifest. 
manifest = "https://colenda.library.upenn.edu/phalt/iiif/2/" + identifier + "/manifest"

# Get the manifest
r = requests.get(manifest)

# Get the information about all the images for this item as a list 
results = r.json()["sequences"][0]['canvases']

# Create a dictionary to collect each image URL (key) and corresponding label (value) for this item. 
imagesDict = {}

# Iterate over each image in the results list to extract the URL and label for the image, adding it to the lists above
for i in range(len(results)):
    label = results[i]['label']
    resource = results[i]['images'][0]['resource']
    images = resource['@id']
    imagesDict[images] = label 
    
# Display the link to the item in Colenda, with the item-specific URL and the item's title as the hyperlinked text. 
display(HTML('<a href="https://colenda.library.upenn.edu/catalog/{}">{}</a>'.format(identifier, year_1881.iloc[count]['metadata.title[1]'])))

# Display the images as a gallery    
gallery(imagesDict, row_height='150px')


## Explore Items by Year and Type

Now that we have a dataframe that combines creation dates with object types, we can look at how the creation of particular object types changes over time. Let's look at the 1880s as an example.

In [22]:
# Create a dataframe containing the years 1880-1889
df_1880s = df_dates_types.loc[(df_dates_types['metadata.year'] > 1879) & (df_dates_types['metadata.year'] < 1890)]

#Create a list of columns to keep
col_list = ['metadata.year', 'metadata.item_type[1]']

# Save the dataframe to contain only the columns in col_list
df_1880s1 = df_1880s[col_list]

# Group the dataframe by the columns Year and Type to calculate the number of elements, and save as column `Count`
df_1880s1 =  df_1880s1.groupby(['metadata.year','metadata.item_type[1]']).size().reset_index(name="Count")

# Sort the column `Year` by its values
df_1880s1 = df_1880s1.sort_values(by=['metadata.year'])

df_1880s

,unique_identifier,metadata.title[1],metadata.year,metadata.item_type[1]
9,ark:/81431/p3000023b,"Trade card; Erichs'; New York, New York, Unite...",1881,Trade cards
14,ark:/81431/p30000507,"Billhead; Jacob Hyman & Son; New York, New Yor...",1886,Billheads
19,ark:/81431/p30157,"Billhead; Jacobs, M.; M. Jacobs & Co.; Phoenix...",1884,Billhead
20,ark:/81431/p3016m,"Billhead; Lichtenberg, J. M.; M. J. Lichtenber...",1883,Billhead
22,ark:/81431/p3019r,"Billhead; Karsky, M.; M. Karsky; Eureka, Nevad...",1885,Billhead
...,...,...,...,...
9393,ark:/81431/p3zq4f,"Billhead; Mendel, Henry; Smith, Mendel & Co.; ...",1885,Billheads
9395,ark:/81431/p3zq8z,"Catalog; ; Frank Bros. & Co.; Philadelphia, Pe...",1886,Catalogs
9396,ark:/81431/p3zr08,"Invitation; Reis, Henry M.; Academy of Music; ...",1885,Invitations
9401,ark:/81431/p3zs2kj50,"Billhead; P. Morgenstern; New York, New York, ...",1885,Billheads


In [23]:
# We have to rename the columns to remove the `.` or else the charts below won't work
df_1880s = df_1880s.rename(columns={"metadata.year": "year", "metadata.item_type[1]": "additionalType"})

In [24]:
# Create a stacked bar chart
alt.Chart(df_1880s).mark_bar(size=3).encode(
    
    # Year on the X axis
    x=alt.X('year:Q', axis=alt.Axis(format='c', title='Year')),
    
    # Number of objects on the Y axis
    y=alt.Y('count()', title='Number of objects'),
    
    # Color according to the type
    color='additionalType:N',
    
    # Details on hover
    tooltip=[alt.Tooltip('additionalType:N', title='Type'), alt.Tooltip('year:Q', title='Year'), alt.Tooltip('count():Q', title='Objects', format=',')]
).properties(width=700)

alt.Chart(...)

Let's try another way of charting changes in the creation of the most common object types over time.

First we'll get the top ten object types (which have years) as a list.

In [25]:
# Get most common 10 values and convert to a list
top_types = df_dates_types['metadata.item_type[1]'].value_counts()[:10].index.to_list()
top_types

['Letters',
 'Billheads',
 'Periodicals',
 'Billhead',
 'Receipts',
 'Trade cards',
 'Letterheads',
 'Envelopes',
 'Pamphlets',
 'Monetary']

Now we'll use the list of `top_types` to filter the creation dates, so we only have events relating to those types of items.

In [26]:
# Only include records where the additionalType value is in the list of top_types
df_top_types = df_dates_types.loc[(df_dates_types['metadata.year'] > 0) & (df_dates_types['metadata.item_type[1]'].isin(top_types))]

In [27]:
# Get the counts for year / type
top_type_counts = df_top_types.groupby('metadata.year')['metadata.item_type[1]'].value_counts().to_frame()
top_type_counts.columns = ['Count']
top_type_counts.reset_index(inplace=True)
print(top_type_counts)

     metadata.year metadata.item_type[1]  Count
0             1680             Pamphlets      5
1             1708             Pamphlets      1
2             1726           Periodicals      1
3             1728           Periodicals      2
4             1732              Receipts      1
..             ...                   ...    ...
656           1898             Envelopes      2
657           1898           Trade cards      2
658           1899           Trade cards      6
659           1905           Trade cards      5
660           1905             Billheads      1

[661 rows x 3 columns]


To chart this data, we're going to use circles for each point and create 'bubble lines' for each item type to show how the number of items collected for that type varied year by year.

In [28]:
# Rename the columns to remove the `.` - formulas do not work without them
top_type_counts = top_type_counts.rename(columns={"metadata.year": "year", "metadata.item_type[1]": "additionalType"})

In [29]:
# Create a chart
alt.Chart(top_type_counts).mark_circle(
    
    # Style the circles
    opacity=0.8,
    stroke='black',
    strokeWidth=1
).encode(
    
    # Year on the X axis
    x=alt.X('year:Q', axis=alt.Axis(format='c', title='Year', labelAngle=0), scale=alt.Scale(zero=False)),
    
    # Object type on the Y axis
    y=alt.Y('additionalType:N', title='Item Type'),
    
    # Size of the circles represents the number of objects
    size=alt.Size('Count:Q',
        scale=alt.Scale(range=[0, 2000]),
        legend=alt.Legend(title='Number of Items')
    ),
    
    # Color the circles by object type
    color=alt.Color('additionalType:N'),
    
    # Provide type, year, and count details on hover
    tooltip=[alt.Tooltip('additionalType:N', title='Type'), alt.Tooltip('year:Q', title='Year'), alt.Tooltip('Count:Q', title='Number of Items', format=',')]
).properties(
    width=700
)

alt.Chart(...)

This chart shows us how item types are distributed over time. Let's calculate the earliest year for each item type in the collection and save it to a CSV for review.

In [30]:
# NumPy is a library for adding support for manipulating and operating on large, multi-dimensional arrays
import numpy as np

# Group DataFrame by the `additionalType` columns and aggregate the year column for its minimum and maximum year
top_type_counts_grouped = top_type_counts.groupby('additionalType').agg({'year' : [np.min, np.max]})

# Write the top_type_counts_grouped dataframe to a comma-separated values (csv) file.
top_type_counts_grouped.to_csv('data/colenda_item_type_years.csv', index=False)

# Display a link to the CSV.
display(FileLink('data/colenda_item_type_years.csv'))

/Users/estene/Documents/GitHub/collections-as-data-notebooks/data/colenda_item_type_years.csv

## Explore Items by Year and Location

In preparation for our the next Jupyter notebook, let's take a look atlocation data instead of item types. To chart this data we're going to use circles for each point and create 'bubble lines' for each state to show how the number of items varies over time.

In [31]:
# Use the function to split the values of the `metadata.geographic_subject` column and expand so that the new dataframe has one split value per row.
df_places = tidy_concat(df_dates, 'metadata.geographic_subject', sep='|')


In [32]:
# Filter the dataframe to only include non-null values
df_places = df_places[df_places['metadata.geographic_subject'].notna()]

# Filter the dataframe to only include geographic subjects that start with `United States`
df_usa = df_places[df_places['metadata.geographic_subject'].str.startswith("United States")]

# Filter the dataframe to only include values that have two dashes (drop anything more detailed than the State level)
df_usa = df_usa[df_usa["metadata.geographic_subject"].str.count("-")==2]

# Filter the dataframe to only include values that are not zero
df_usa = df_usa.loc[df_usa["metadata.year"] != 0]

# Rename the columns to remove the `.` or else the charts below won't work
df_usa = df_usa.rename(columns={'metadata.geographic_subject': 'geographic_subject','metadata.year':'year'})

# Get the counts for year / geographics_subject
df_usa_counts = df_usa.groupby('year')['geographic_subject'].value_counts().to_frame()
df_usa_counts.columns = ['Count']
df_usa_counts.reset_index(inplace=True)
print(df_usa_counts)

      year              geographic_subject  Count
0     1726       United States -- New York      1
1     1732  United States -- Massachusetts      1
2     1737       United States -- New York      1
3     1740  United States -- New Hampshire      1
4     1743   United States -- Rhode Island      1
...    ...                             ...    ...
1320  1899       United States -- New York      1
1321  1905     United States -- California      4
1322  1905       United States -- New York      2
1323  1905           United States -- Iowa      1
1324  1905          United States -- Maine      1

[1325 rows x 3 columns]


In [33]:
# Create a chart
alt.Chart(df_usa_counts).mark_circle(
    
    # Style the circles
    opacity=0.8,
    stroke='black',
    strokeWidth=1
).encode(
    
    # Year on the X axis
    x=alt.X('year:Q', axis=alt.Axis(format='c', title='Year', labelAngle=0), scale=alt.Scale(zero=False)),
    
    # Object type on the Y axis
    y=alt.Y('geographic_subject:N', title='State'),
    
    # Size of the circles represents the number of objects
    size=alt.Size('Count:Q',
        scale=alt.Scale(range=[0, 2000]),
        legend=alt.Legend(title='Number of Items')
    ),
    
    # Color the circles by object type
    #color=alt.Color('geographic_subject:N'),
    
    # Provide state, year, and count details on hover
    tooltip=[alt.Tooltip('geographic_subject:N', title='State'), alt.Tooltip('year:Q', title='Year'), alt.Tooltip('Count:Q', title='Number of Items', format=',')]
).properties(
    width=700
)

alt.Chart(...)

# Need Help?
<div class="alert alert-block alert-warning">
    <p>For additional Python and Digital Scholarship resources:</p>
    <ul>
        <li><a href"https://www.w3schools.com/python/pandas/default.asp">Pandas Tutorial from W3 Schools</a></li>
        <li><a href"https://altair-viz.github.io/altair-tutorial/README.html">Altair Tutorial from W3 Schools</a></li>
        <li><a href="https://guides.library.upenn.edu/digital-scholarship">Center for Research Data and Digital Scholarship</a></li>
    </ul>
    <p>For help with this notebook:</p>    
<ul>
    <li>If you encounter any errors in this notebook, you can open an issue on GitHub or email estene@upenn.edu and reference this notebook.</li>

<li>If you encounter any errors while working with the collection metadata (an incorrect date or broken ARK identifier), you can email estene@upenn.edu.</li>

<li>Colenda is still a beta service. If you encounter issues with accessing any of the IIIF images or links, visit
    <a href="https://colenda.library.upenn.edu/">Colenda</a></li>
    </ul>
</div>

----

# Credits

Created by [Emily Esten](https://www.library.upenn.edu/people/staff/emily-esten). 

Judaica Digital Humanities at the <a href="http://library.upenn.edu">Penn Libraries</a> (also referred to as Judaica DH) is a robust program of projects and tools for experimental digital scholarship with Judaica collections, informed by digital humanities, Jewish studies, and cultural heritage approaches. Visit our [website](judaicadh.library.upenn.edu).

The pre-harvested dataset for this notebook works with items from the **Arnold and Deanne Kaplan Collection of Early American Judaica**. Donated to the University of Pennsylvania Libraries in 2012 by the Kaplans, and growing each year, this collection teaches us about the everyday lives, families, communal institutions, religious organizations, voluntary associations,  businesses, and political circumstances of Jewish life throughout the western hemisphere over four centuries. More information about the collection can be found at [https://kaplan.exhibits.library.upenn.edu](https://kaplan.exhibits.library.upenn.edu). 

This notebook references existing code and Jupyter notebooks, including: 
* [GLAM Workbench for the National Museum of Australia](https://doi.org/10.5281/zenodo.3544747) sponsored by the [Humanities, Arts and Social Sciences (HASS) Data Enhanced Virtual Lab](https://tinker.edu.au/).
* [Library of Congress Data Exploration: IIIF](https://github.com/LibraryOfCongress/data-exploration/blob/26510c3f4da0bc85dfa87e82141173b1830e9d64/IIIF.ipynb).
* Gustavo Candela, María Dolores Sáez, Pilar Escobar, Manuel Marco-Such, & Rafael C.Carrasco. (2020, May 8). hibernator11/notebook-iiif-images: release1.1 (Version 1.1). Zenodo. [http://doi.org/10.5281/zenodo.3816611](https://zenodo.org/badge/latestdoi/255172461). 
* [Genes for Project Cognoma](https://github.com/cognoma/genes/blob/721204091a96e55de6dcad165d6d8265e67e2a48/2.process.py)


This work is modeled after Tim Sherrat's work with the National Museum of Australia, which was issued under an MIT Licesnse. 